#C6W5 TAXI SHITKOV

###1. Создать 6 наборов признаков для каждой из задач регрессии.

* идентификатор географической зоны — категориальный;
* год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно и так, и так;
* синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты;
* количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−Ky_T. Начать с K = 6;
* количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd. Начать с Kd = 2;
* суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц;
* *значения прогнозов SARIMAX(нужно считать на предыдущие месяцы, это 10 часов на 1 месяц). Это очень затратно по вычислению, так что мимо.*

In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os

In [0]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
#список регионов из C6W2:
#Загрузить агрегированные данные за май 2016:
filepath = '/content/gdrive/My Drive/ml/c6/data/'
filename = '2016-5.csv'
data = pd.read_csv(filepath + filename, index_col = 0)
data_arr = data.to_numpy()
data_mean = np.asarray([data_arr.T[i].mean() for i in range(len(data_arr.T))])

In [0]:
#Список ячеек для удаления из датасета:
rem = np.asarray([-1 if (data_mean > 5)[i] == True else i for i in range(len(data_mean))])
rem[rem >= 0]
rem = data.columns[rem]
rem_list = list(rem)

In [0]:
#Список регионов для работы
ind = np.asarray([i if (data_mean >= 5)[i] == True else 0 for i in range(len(data_mean))])
ind = ind[ind > 0]
ind = data.columns[ind]
cells = list(ind)

In [0]:
#последние N месяцев
import os

N = 6

path = '/content/gdrive/My Drive/ml/c6/data/'
fl = []
for filename in os.listdir(path):
  fl.append(filename)
  
fl.sort()

fl = fl[-(N + 1):-1]

print(fl)

['2016-1.csv', '2016-2.csv', '2016-3.csv', '2016-4.csv', '2016-5.csv', '2016-6.csv']


In [0]:
from datetime import datetime

In [0]:
def make_data(fl, rem, path):  
  #загрузить данные
  month_arr = []
  for m in range(len(fl)):
    filename = str(fl[m])
    data = pd.read_csv(filepath + filename, index_col = 0)
    month_arr.append(pd.DataFrame(data, index = [datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in data.index]))

  data = pd.concat(month_arr)

  data.drop(rem, axis='columns', inplace=True)
  
  return data

data = make_data(fl, rem, path)
data.head()

,region_1075,region_1076,region_1077,region_1125,region_1126,region_1127,region_1128,region_1129,region_1130,region_1131,region_1132,region_1172,region_1173,region_1174,region_1175,region_1176,region_1177,region_1178,region_1179,region_1180,region_1181,region_1182,region_1183,region_1184,region_1221,region_1222,region_1223,region_1224,region_1225,region_1227,region_1228,region_1229,region_1230,region_1231,region_1232,region_1233,region_1234,region_1235,region_1272,region_1273,...,region_1376,region_1377,region_1378,region_1380,region_1382,region_1383,region_1384,region_1385,region_1386,region_1387,region_1388,region_1389,region_1390,region_1426,region_1431,region_1434,region_1435,region_1436,region_1437,region_1438,region_1439,region_1441,region_1442,region_1480,region_1482,region_1483,region_1530,region_1532,region_1533,region_1580,region_1630,region_1684,region_1733,region_1734,region_1783,region_2068,region_2069,region_2118,region_2119,region_2168
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,35.0,16.0,39.0,31.0,29.0,40.0,548.0,649.0,759.0,930.0,695.0,645.0,323.0,24.0,26.0,34.0,25.0,57.0,14.0,420.0,782.0,1001.0,1273.0,938.0,403.0,664.0,519.0,213.0,51.0,49.0,...,55.0,51.0,27.0,23.0,22.0,260.0,908.0,329.0,56.0,93.0,58.0,53.0,72.0,42.0,52.0,5.0,20.0,70.0,66.0,38.0,30.0,30.0,23.0,18.0,42.0,38.0,20.0,25.0,39.0,13.0,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,51.0,38.0,91.0,71.0,29.0,34.0,588.0,534.0,518.0,671.0,830.0,645.0,338.0,22.0,67.0,88.0,57.0,59.0,11.0,613.0,828.0,1116.0,1514.0,830.0,420.0,567.0,663.0,271.0,93.0,86.0,...,163.0,107.0,69.0,40.0,15.0,312.0,990.0,406.0,91.0,156.0,113.0,64.0,106.0,78.0,51.0,7.0,44.0,127.0,133.0,56.0,45.0,53.0,62.0,37.0,92.0,64.0,53.0,64.0,130.0,23.0,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,21.0,31.0,75.0,78.0,22.0,56.0,612.0,431.0,401.0,503.0,683.0,530.0,374.0,13.0,82.0,82.0,55.0,68.0,25.0,615.0,763.0,962.0,1363.0,739.0,484.0,705.0,541.0,214.0,100.0,93.0,...,183.0,130.0,101.0,39.0,10.0,235.0,922.0,269.0,61.0,112.0,127.0,78.0,112.0,91.0,50.0,1.0,29.0,101.0,107.0,52.0,54.0,33.0,51.0,67.0,129.0,63.0,57.0,125.0,137.0,35.0,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,10.0,28.0,56.0,63.0,9.0,56.0,711.0,394.0,391.0,523.0,587.0,715.0,410.0,15.0,49.0,85.0,49.0,51.0,7.0,542.0,782.0,968.0,1202.0,771.0,742.0,615.0,263.0,152.0,52.0,55.0,...,188.0,115.0,79.0,25.0,5.0,152.0,550.0,157.0,51.0,110.0,98.0,63.0,89.0,90.0,43.0,6.0,19.0,59.0,85.0,44.0,42.0,44.0,44.0,47.0,94.0,34.0,55.0,86.0,124.0,34.0,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,5.0,8.0,22.0,22.0,3.0,47.0,536.0,506.0,388.0,381.0,403.0,550.0,256.0,4.0,29.0,23.0,30.0,53.0,2.0,444.0,703.0,646.0,647.0,560.0,701.0,379.0,150.0,62.0,26.0,29.0,...,135.0,57.0,34.0,24.0,4.0,95.0,351.0,96.0,32.0,56.0,65.0,48.0,74.0,76.0,37.0,0.0,17.0,73.0,70.0,27.0,40.0,41.0,54.0,50.0,83.0,46.0,45.0,81.0,113.0,26.0,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


In [0]:
from datetime import datetime
def make_features(data, k_features, k_poly):
  col = []

  for i in range(1, k_features + 1):
    col.append(str('sin_' + str(i)))
    col.append(str('cos_' + str(i)))
  for i in range(1, k_poly + 1):
    col.append(str('poly_' + str(i)))
    
  features = pd.DataFrame(columns = col, index = data.index)
  
  features['num'] = [int(i) for i in range(1, len(data) + 1)]
  
  for k in range(1, k_features + 1):
    features[str('sin_' + str(k))] = [np.sin(features.num[x] * 2 * np.pi * k / 168) for x in range(len(data))]
    features[str('cos_' + str(k))] = [np.cos(features.num[x] * 2 * np.pi * k / 168) for x in range(len(data))]

  for k in range(1, k_poly + 1):
    features[str('poly_' + str(k))] = [features.num[x] ** k  for x in range(len(data))]  

  features['year'] = [data.index[i].year for i in range(len(data))]
  features['month'] = [data.index[i].month for i in range(len(data))]
  features['day'] = [data.index[i].day for i in range(len(data))]
  features['weekday'] = [data.index[i].weekday() for i in range(len(data))]
  features['hour'] = [data.index[i].hour for i in range(len(data))]
  features.drop(['num'], axis='columns', inplace=True)
  
  return features

In [0]:

#Количество создаваемых признаков sin, cos, poly
k_features = 10
k_poly = 2

fl = []
filepath = '/content/gdrive/My Drive/ml/c6/c6w5/regions/'
for filename in os.listdir(filepath):
  fl.append(filename)
if 'features.csv' not in fl:
  print('features calculaing: WORK')
  features = make_features(data, k_features, k_poly)
  features.to_csv('/content/gdrive/My Drive/ml/c6/c6w5/regions/' + 'features.csv')
  print('features calculaing: DONE')
else:
  features = pd.read_csv('/content/gdrive/My Drive/ml/c6/c6w5/regions/' + 'features.csv', index_col = 0)
  print('features calculaing: DONE')

features calculaing: DONE


In [0]:
fl = []
filepath = '/content/gdrive/My Drive/ml/c6/c6w5/regions/'
for filename in os.listdir(filepath):
  reg = 'region_' + filename[:-4]
  fl.append(reg)

K  = 12
Kd = 4

sl = [12, 24, 168, 672]

df = []
for i in range(len(cells)):
  #создать датафрейм по региону
  df.append(pd.DataFrame(data[cells[i]]))
  #проверка наличия датасета
  if cells[i] not in fl:
    print(str(cells[i]) + ': WORK')
    #переименовать столбец
    df[i].columns = ['yT']
    #добавить значение количества поездок за предыдущие K часов
    for j in range(1, K + 1):
      df[i][str('yT-' + str(j))] = df[i][df[i].columns[0]].shift(j)
    #добавить значение количества поездок за предыдущие dK часов * 24
    for j in range(1, Kd + 1):
      df[i][str('yT-' + str(j * 24))] = df[i][df[i].columns[0]].shift(24 * j)
    #добавить сумму поездок за предыдущие sl[i] часов
    for j in sl:
      df[i]['sum_' + str(j)] = [np.nan for l in range(j)] + [sum([df[i][df[i].columns[0]][n] for n in range(k, k + j)]) for k in range(len(df[i]) - j)]
    #добавить категориальный признак region
    df[i]['region'] = [cells[i][-4:] for n in range(len(df[i]))]
    df[i]['region'] = df[i]['region'].astype('category')
    #склеить текущий датасет с features
    df[i] = pd.concat([df[i], features], axis=1)
    #добавить отклик на +1...+6 часов
    for j in range(1, 7):
      df[i][str('yT+' + str(j))] = df[i][df[i].columns[0]].shift(-j)
    #сохранить данные на диск
    df[i].to_csv('/content/gdrive/My Drive/ml/c6/c6w5/regions/' + str(cells[i][-4:]) + '.csv')
    print(str(cells[i]) + ': DONE')

In [0]:
fl = []
filepath = '/content/gdrive/My Drive/ml/c6/c6w5/regions2/'
for filename in os.listdir(filepath):
  reg = 'region_' + filename[:-4]
  fl.append(reg)

K  = 23
Kd = 7

sl = [12, 24, 168, 672]

df = []
for i in range(len(cells)):
  #создать датафрейм по региону
  df.append(pd.DataFrame(data[cells[i]]))
  #проверка наличия датасета
  if cells[i] not in fl:
    print(str(cells[i]) + ': WORK')
    #переименовать столбец
    df[i].columns = ['yT']
    #добавить значение количества поездок за предыдущие K часов
    for j in range(1, K + 1):
      df[i][str('yT-' + str(j))] = df[i][df[i].columns[0]].shift(j)
    #добавить значение количества поездок за предыдущие dK часов * 24
    for j in range(1, Kd + 1):
      df[i][str('yT-' + str(j * 24))] = df[i][df[i].columns[0]].shift(24 * j)
    #добавить сумму поездок за предыдущие sl[i] часов
    for j in sl:
      df[i]['sum_' + str(j)] = [np.nan for l in range(j)] + [sum([df[i][df[i].columns[0]][n] for n in range(k, k + j)]) for k in range(len(df[i]) - j)]
    #добавить категориальный признак region
    df[i]['region'] = [cells[i][-4:] for n in range(len(df[i]))]
    df[i]['region'] = df[i]['region'].astype('category')
    #склеить текущий датасет с features
    df[i] = pd.concat([df[i], features], axis=1)
    #добавить отклик на +1...+6 часов
    for j in range(1, 7):
      df[i][str('yT+' + str(j))] = df[i][df[i].columns[0]].shift(-j)
    #сохранить данные на диск
    df[i].to_csv('/content/gdrive/My Drive/ml/c6/c6w5/regions2/' + str(cells[i][-4:]) + '.csv')
    print(str(cells[i]) + ': DONE')

In [0]:
#считать данные
filepath = '/content/gdrive/My Drive/ml/c6/c6w5/regions2/'
fl = []
for filename in os.listdir(filepath):
  fl.append(filename)
df = []  
for i in range(len(fl)):
  df.append(pd.read_csv(filepath + fl[i], index_col = 0))

#склеить датафреймы  
data = pd.concat(df, sort=False)
#заменить признак region по OneHot
data = pd.concat([data, pd.get_dummies(data.region, prefix = 'reg')], axis = 1)


In [0]:
#заменить признак weekday по OneHot
data = pd.concat([data, pd.get_dummies(data.weekday, prefix = 'weekday')], axis = 1)
#удалить столбец "weekday"
data.drop(['weekday'], axis='columns', inplace=True)

In [0]:
#заменить признак hour по OneHot
data = pd.concat([data, pd.get_dummies(data.hour, prefix = 'hour')], axis = 1)
#удалить столбец "hour"
data.drop(['hour'], axis='columns', inplace=True)

###2. Разбейте каждую из шести выборок на три части:

*    обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
*    тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
*    итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

In [0]:
#Обучающая выборка: train_data
data_train = data[data.index < '2016-04-30 18:00:00'].dropna()
#Тестовая выборка : test_data
data_test = data[(data.index < '2016-05-31 18:00:00') & (data.index >= '2016-04-30 23:00:00')].dropna()
#Контрольная выборка: control_data
data_control = data[data.index >= '2016-05-31 23:00:00'].dropna()

In [0]:
#целевые метки
target_labels = []
for i in range(1, 7):
  target_labels.append(str('yT+' + str(i)))

In [0]:
reg_col  = ['reg_1075', 'reg_1076', 'reg_1077', 'reg_1125', 'reg_1126', 'reg_1127', 'reg_1128', 'reg_1129', 'reg_1130', 'reg_1131', 'reg_1132', 'reg_1172', 'reg_1173', 'reg_1174', 'reg_1175', 'reg_1176', 'reg_1177', 'reg_1178', 'reg_1179', 'reg_1180', 'reg_1181', 'reg_1182', 'reg_1183', 'reg_1184', 'reg_1221', 'reg_1222', 'reg_1223', 'reg_1224', 'reg_1225', 'reg_1227', 'reg_1228', 'reg_1229', 'reg_1230', 'reg_1231', 'reg_1232', 'reg_1233', 'reg_1234', 'reg_1235', 'reg_1272', 'reg_1273', 'reg_1274', 'reg_1278', 'reg_1279', 'reg_1280', 'reg_1281', 'reg_1282', 'reg_1283', 'reg_1284', 'reg_1285', 'reg_1286', 'reg_1287', 'reg_1326', 'reg_1327', 'reg_1331', 'reg_1332', 'reg_1333', 'reg_1334', 'reg_1335', 'reg_1336', 'reg_1337', 'reg_1338', 'reg_1339', 'reg_1376', 'reg_1377', 'reg_1378', 'reg_1380', 'reg_1382', 'reg_1383', 'reg_1384', 'reg_1385', 'reg_1386', 'reg_1387', 'reg_1388', 'reg_1389', 'reg_1390', 'reg_1426', 'reg_1431', 'reg_1434', 'reg_1435', 'reg_1436', 'reg_1437', 'reg_1438', 'reg_1439', 'reg_1441', 'reg_1442', 'reg_1480', 'reg_1482', 'reg_1483', 'reg_1530', 'reg_1532', 'reg_1533', 'reg_1580', 'reg_1630', 'reg_1684', 'reg_1733', 'reg_1734', 'reg_1783', 'reg_2068', 'reg_2069', 'reg_2118', 'reg_2119', 'reg_2168']
y_time   = ['year','month','day','yT','yT-1','yT-2','yT-3','yT-4','yT-5','yT-6','yT-7','yT-8','yT-9','yT-10','yT-11','yT-12','yT-13','yT-14','yT-15','yT-16','yT-17']
hour_col = ['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23']

In [0]:
X_train = data_train.copy()
X_train.drop(target_labels + reg_col + y_time + hour_col, axis='columns', inplace=True)
#удалить столбец "region"
X_train.drop(['region'], axis='columns', inplace=True)

X_test = data_test.copy()
X_test.drop(target_labels + reg_col + y_time + hour_col, axis='columns', inplace=True)
#удалить столбец "region"
X_test.drop(['region'], axis='columns', inplace=True)

In [0]:
i = 0
y_train = pd.DataFrame(data_train[target_labels[i]]).values
y_test  = pd.DataFrame(data_test[target_labels[i]]).values

In [0]:
#Нормализовать признаки
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

###3. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016.
Желательно, чтобы модель:

* допускала попарные взаимодействия между признаками.
* была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы L1 и L2).

* https://nagornyy.me/courses/data-science/regression/
* https://habr.com/ru/company/ods/blog/323890/
* https://nuancesprog.ru/p/2922/

In [0]:
%%time
from sklearn.linear_model import Lasso

alph = [0.1, 0.3, 0.5, 1, 2, 3, 4, 5]

best_err = inf
best_alph = inf
i = 0

for a in alph:
  lasso = Lasso(alpha=a)
  lasso.fit(X_train_scaled, y_train)
  y_test_pred = lasso.predict(scaler.transform(X_test))
  #Ошибка на мае
  current_err = sum(abs(data_test[target_labels[i]].values - y_test_pred))/len(y_test_pred)
  print(a, current_err)
  if current_err < best_err:
    best_err = current_err
    best_alph = a
print('BEST ALPHA:', best_alph)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':


(0.1, 30.056333906549625)
(0.3, 30.074392403247423)
(0.5, 29.809649271593596)
(1, 29.558667507194514)
(2, 29.04932198257793)
(3, 28.901132092318527)
(4, 29.031147408107657)
(5, 29.171252950011922)
('BEST ALPHA:', 3)
CPU times: user 1min 5s, sys: 2.17 s, total: 1min 7s
Wall time: 1min 4s


In [0]:
%%time
from sklearn.linear_model import Lasso

alph = [5]

best_err = inf
best_alph = inf
i = 0

for a in alph:
  lasso = Lasso(alpha=a)
  lasso.fit(X_train_scaled, y_train)
  y_test_pred = lasso.predict(scaler.transform(X_test))
  #Ошибка на мае
  current_err = sum(abs(data_test[target_labels[i]].values - y_test_pred))/len(y_test_pred)
  print(a, current_err)
  if current_err < best_err:
    best_err = current_err
    best_alph = a
print('BEST ALPHA:', best_alph)

(5, 29.171252950011922)
('BEST ALPHA:', 5)
CPU times: user 2.45 s, sys: 258 ms, total: 2.71 s
Wall time: 2.41 s


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':


In [0]:
lasso = Lasso(alpha=0.5)
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
i = 0
y_train = pd.DataFrame(data_train[target_labels[i]])
lasso = Lasso(alpha=0.5)
lasso.fit(X_train_scaled, y_train)
y_train_pred = lasso.predict(X_train_scaled)
current_err = sum(abs(data_train[target_labels[i]].values - y_train_pred))/len(y_train_pred)
current_err

29.563713734638817

In [0]:
zip(X_test.columns[lasso.coef_ > 0], lasso.coef_[lasso.coef_ > 0])

[('yT-19', 1.7142521110527695),
 ('yT-22', 45.101749962185146),
 ('yT-23', 101.53038920033748),
 ('yT-144', 25.99482148739954),
 ('yT-168', 136.1484319896912),
 ('sum_672', 0.622083897750811),
 ('cos_1', 0.6013787496760077),
 ('sin_2', 3.240660955819526),
 ('sin_4', 2.772081635743124),
 ('cos_4', 2.2854015630728997),
 ('cos_6', 1.3080654911073073),
 ('cos_8', 1.8913484684892745),
 ('sin_9', 4.429076257109779),
 ('cos_9', 0.18341072794274224),
 ('weekday_3', 0.5678314783385225)]

###4.1. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд.

###4.2. Посчитайте в ноутбуке ошибку прогноза Q_may.

###4.3. Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

In [0]:
%%time
from sklearn.linear_model import Lasso

alph = [0.5, 0.5, 0.5, 0.3, 0.1, 0.1]

lasso = []
err   = []
regr  = []


for i in range(6):
  y_train = pd.DataFrame(data_train[target_labels[i]]).values
  y_test  = pd.DataFrame(data_test[target_labels[i]]).values
  
  lasso.append(Lasso(alpha = 0.1))
  lasso[i] = Lasso(alpha=alph[i])
  lasso[i].fit(X_train_scaled, y_train)
  y_test_pred = lasso[i].predict(scaler.transform(X_test))
  y_test_pred = y_test_pred * (y_test_pred > 0)
  current_err = sum(abs(data_test[target_labels[i]].values - y_test_pred))/len(y_test_pred)
  print(i, alph[i], current_err)
  err.append(current_err)
  regr.append(lasso[i])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


(0, 0.5, 28.308497962551446)
(1, 0.5, 32.352915349354085)
(2, 0.5, 34.5552609314065)
(3, 0.3, 35.52358727495987)
(4, 0.1, 35.6679210543814)
(5, 0.1, 36.169326433970376)
CPU times: user 2min 8s, sys: 1.69 s, total: 2min 10s
Wall time: 2min 8s


In [0]:
for i in range(len(regr)):
  print(zip(X_test.columns[regr[i].coef_ > 0], regr[i].coef_[regr[i].coef_ > 0]))

[('yT-19', 1.7142521110527695), ('yT-22', 45.101749962185146), ('yT-23', 101.53038920033748), ('yT-144', 25.99482148739954), ('yT-168', 136.1484319896912), ('sum_672', 0.622083897750811), ('cos_1', 0.6013787496760077), ('sin_2', 3.240660955819526), ('sin_4', 2.772081635743124), ('cos_4', 2.2854015630728997), ('cos_6', 1.3080654911073073), ('cos_8', 1.8913484684892745), ('sin_9', 4.429076257109779), ('cos_9', 0.18341072794274224), ('weekday_3', 0.5678314783385225)]
[('yT-21', 28.679697070129414), ('yT-22', 150.81545746401545), ('yT-23', 13.559064341764437), ('yT-144', 11.372562739603753), ('yT-168', 108.03917630936662), ('sum_672', 31.758735600939403), ('sin_1', 0.23868366172613925), ('sin_2', 0.4705260254024182), ('cos_3', 1.69824360220985), ('sin_4', 3.0935098513849413), ('cos_4', 1.1411721128990127), ('cos_6', 3.1303925582022822), ('cos_8', 1.7951251915014665), ('sin_9', 4.72065661037411), ('cos_9', 2.9545595602412216), ('weekday_4', 1.7397074058637356)]
[('yT-20', 29.479330831233007

In [0]:
#Ошибка за май на LASSO
print(sum(err)/6)

33.762918167770614


###5. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате yyyy.mm.dd, histEndHour — час конца истории от 0 до 23, step ∈{1,2,3,4,5,6} — номер отсчёта, на который вы делаете предсказание (1-6); столбец y — ваш прогноз.

In [0]:
X_ctrl = data_control.copy()
X_ctrl.drop(target_labels + reg_col + y_time + hour_col, axis='columns', inplace=True)
#удалить столбец "region"
X_ctrl.drop(['region'], axis='columns', inplace=True)

In [0]:
predict = []
for i in range((len(target_labels))):
  current = regr[i].predict(scaler.transform(X_ctrl))
  current = current * (current > 0)
  predict.append(current)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
df_list = []
id_list = []
for i in range((len(target_labels))):
  df = pd.DataFrame(columns = ['id', 'y'])
  id_list.append(list(map(lambda x, y : str(x) + str('_') + str(y)[:10] + str('_') + str(int(str(y)[11:13])) + str('_') + str(i + 1), data_control.region, data_control.index)))
  df.id = id_list[i]
  df.y = predict[i]
  df_list.append(df)

In [0]:
data_final = pd.concat(df_list)

In [0]:
data_final.to_csv('/content/gdrive/My Drive/ml/c6/c6w5/final.csv', index=False)

In [0]:
data_final.head()

,id,y
0,1075_2016-05-31_23_1,27.764118
1,1075_2016-06-01_0_1,14.486390
2,1075_2016-06-01_1_1,2.066418
3,1075_2016-06-01_2_1,-0.000000
4,1075_2016-06-01_3_1,-0.000000


###6. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

125 Konstantin Shitkov 30.93866 https://inclass.kaggle.com/submissions/12330337/12330337.raw